# Voltage clamp in a neuron with Hodgkin & Huxley conductances

 

## Step 1: Setup

In [ ]:
# Setup inline plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# For Google Colab, this line installs NEURON
!pip install neuron quantities

In [ ]:
# Fetch mechanisms
# Uncomment this line if on google colab
#!git clone https://github.com/ABL-Lab/NSC6084-A25.git

In [ ]:
# Compile the mechanisms
# Note: recompiled mechanisms will not take effect until neuron is imported or the jupyter kernel is restarted

# Uncomment this line if on google colab
#!nrnivmodl ./NSC6084-A25/Sept16/mechanisms
# Uncomment this line if running locally
!nrnivmodl mechanisms

In [ ]:
# We will let this library handle unit conversion for us
import quantities as pq
from quantities import um, nS, mV, cm, ms, nA, S, uF, Hz, degrees, s, MOhm, mS, mm

In [ ]:
# Import and initialize NEURON
import neuron
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# Import other modules we need
import numpy as np

## Step 2: Define the circuit
We will use a single compartment, called a "Section" (more on that in next lectures). <br>
It has a cylindrical geometry with length "L" and a diameter "diam", and a specific capacitance "cm" (capacitance per area) <br>
**Unit conversion is a common source of error, so we will be explicit with our units.** 

In [ ]:
soma = h.Section()

### Query NEURON for the expected units for soma.L & soma.diam

In [ ]:
[h.units(x) for x in ["L", "diam"]]

In [ ]:
soma.L = 10 * um
soma.diam =  10 * um

In [ ]:
volume = soma(0.5).volume() * um**3

In [ ]:
area = soma(0.5).area() * um**2

In [ ]:
area

In [ ]:
volume

### Assign the membrane capacitance "everywhere"

In [ ]:
h.units("cm")  # Query the expected units

In [ ]:
specific_membrane_capacitance = 1 * uF/cm**2

In [ ]:
for sec in soma.wholetree():
    sec.cm = specific_membrane_capacitance #  specific membrane capacitance (micro Farads / cm^2)
    sec.Ra = 100

### Add the Hodgkin-Huxley conductances

In [ ]:
# This model includes the transient Na+, persistent K+ and the leak conductances
soma.insert("hh")

That's almost too easy!

### Parametize the leak conductance G = 1/R

In [ ]:
G = 1 * nS  # R = 1/G in our RC circuit

In [ ]:
v_rest = -70*mV

In [ ]:
tau_m = (specific_membrane_capacitance * area / G).rescale(ms)

In [ ]:
tau_m

In [ ]:
# Assign the leak conductance everywhere
for seg in soma:
    seg.hh.gl = (G/area).rescale(S/cm**2)  # Compute specific conductance, and rescale to units of 'S/cm2'
    seg.hh.el = -54.3

In [ ]:
tau_m = ((soma(0.5).cm * uF/cm**2 )/ (soma(0.5).hh.gl *S/cm**2 )).rescale(ms)

In [ ]:
tau_m

In [ ]:
soma(0.5).hh.gl # comparable to Conner-Stevens gL

In [ ]:
(soma(0.5).hh.gkbar *S/cm**2 ).rescale(mS/mm**2)  # comparable to Conner-Stevens gkbar

In [ ]:
(0.12 *S/cm**2 ).rescale(mS/mm**2) # comparable to Conner-Stevens gnabar

In [ ]:
(1.0*mS/mm**2).rescale(S/cm**2)

### Inspect our parameters

In [ ]:
soma.psection()

### Add voltage clamp

In [ ]:
vclamp = h.SEClamp(soma(0.5))

In [ ]:
vclamp.dur1 = 200 * ms  # Inject current 500ms after the start of the simulation 
vclamp.dur2 = 700 * ms  # stop injecting current at 520ms 
vclamp.amp1 = v_rest
vclamp.amp2 = -40*mV
vclamp.amp3 = v_rest


In [ ]:
(1/G).rescale(MOhm)

In [ ]:
vclamp.rs = 0.01 # The clamp series resistance should be < 1/100 Rin

## Step 3: Run the simulation

### Define recordings of simulation variables

In [ ]:
soma_v = h.Vector().record(soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

In [ ]:
vclamp_i = h.Vector().record(vclamp._ref_i)

### Function to run a holding voltage simulation

In [ ]:
def run_sim(holding_voltage):
    vclamp.amp2 = holding_voltage
    h.finitialize( float(v_rest) )
    h.continuerun( float(1000 * ms) )
    return np.array(t), np.array(soma_v), np.array(vclamp_i)

## Step 4: Plot the results

In [ ]:
soma(0.5).hh.gkbar = 0.0 # Zero HH K conductance
soma(0.5).hh.gnabar = 0.12
fig = plt.figure()
ax1, ax2 = fig.subplots(2, 1)
for holding_voltage in [-70, -60, -50, -40, -30, -20, -10, 0]:
    a_t, v, i = run_sim(holding_voltage)
    ax1.plot(a_t, i, lw=2, label="%f mV" % holding_voltage)
    ax2.plot(a_t, v, lw=2, label="%f mV" % holding_voltage)
    #plt.legend(fontsize=12)
ax1.set_xlabel("t [ms]", size=16)
ax1.set_ylabel("i [nA]", size=16)
ax2.set_xlabel("t [ms]", size=16)
ax2.set_ylabel("v [mV]", size=16)
ax1.axis([195,210,-8,8])
ax2.axis([195,210, -80, 20])

In [ ]:
soma(0.5).hh.gkbar = 0.036
soma(0.5).hh.gnabar = 0.0 # Zero HH Na conductance
fig = plt.figure()
ax1, ax2 = fig.subplots(2, 1)
for holding_voltage in [-70, -60, -50, -40, -30, -20, -10, 0]:
    a_t, v, i = run_sim(holding_voltage)
    ax1.plot(a_t, i, lw=2, label="%f mV" % holding_voltage)
    ax2.plot(a_t, v, lw=2, label="%f mV" % holding_voltage)
    #plt.legend(fontsize=12)
ax1.set_xlabel("t [ms]", size=16)
ax1.set_ylabel("i [nA]", size=16)
ax2.set_xlabel("t [ms]", size=16)
ax2.set_ylabel("v [mV]", size=16)
ax1.axis([195,210,-8,8])
ax2.axis([195,210, -80, 20])

In [ ]:
soma(0.5).hh.gkbar = 0.0
soma(0.5).hh.gnabar = 0.0 # Zero HH Na conductance
fig = plt.figure()
ax1, ax2 = fig.subplots(2, 1)
for holding_voltage in [-70, -60, -50, -40, -30, -20, -10, 0]:
    a_t, v, i = run_sim(holding_voltage)
    ax1.plot(a_t, i, lw=2, label="%f mV" % holding_voltage)
    ax2.plot(a_t, v, lw=2, label="%f mV" % holding_voltage)
    #plt.legend(fontsize=12)
ax1.set_xlabel("t [ms]", size=16)
ax1.set_ylabel("i [nA]", size=16)
ax2.set_xlabel("t [ms]", size=16)
ax2.set_ylabel("v [mV]", size=16)
ax1.axis([195,210,-0.2,0.2])
ax2.axis([195,210, -80, 20])

### *** Intermission - Back to lecture ***

## Part 2 - Add a transient K+ ion channel (IA) as in the Conner & Stevens model (see: Dayan & Abbott, pg 196)

In [ ]:
soma.insert("K_Tst")

In [ ]:
soma(0.5).K_Tst.gK_Tstbar = 0.1*0.477

In [ ]:
soma(0.5).hh.gkbar = 0.0
soma(0.5).hh.gnabar = 0.0 # Zero HH Na conductance
fig = plt.figure()
ax1, ax2 = fig.subplots(2, 1)
for holding_voltage in [-70, -60, -50, -40, -30, -20, -10, 0]:
    a_t, v, i = run_sim(holding_voltage)
    ax1.plot(a_t, i, lw=2, label="%f mV" % holding_voltage)
    ax2.plot(a_t, v, lw=2, label="%f mV" % holding_voltage)
    #plt.legend(fontsize=12)
ax1.set_xlabel("t [ms]", size=16)
ax1.set_ylabel("i [nA]", size=16)
ax2.set_xlabel("t [ms]", size=16)
ax2.set_ylabel("v [mV]", size=16)
ax1.axis([195,225,-0.1,0.6])
ax2.axis([195,225, -80, 20])

## There is also a transient K+ ion channel (IA) model for the genetically expressed ion channel Kv4.2 from channelpedia 

In [ ]:
soma.insert("Kv4_2_0016")

In [ ]:
soma(0.5).Kv4_2_0016.gKv4_2bar = 0.1*0.477
soma(0.5).Kv4_2_0016.q10 = 3.0

In [ ]:
soma(0.5).Kv4_2_0016.q10

In [ ]:
# Zero other transient K+ channel
soma(0.5).K_Tst.gK_Tstbar = 0

In [ ]:
def run_sim2(holding_voltage, kv42_temp=34):
    vclamp.amp2 = holding_voltage
    h.celsius = kv42_temp
    h.finitialize( float(v_rest) )
    h.continuerun( float(1000 * ms) )
    return np.array(t), np.array(soma_v), np.array(vclamp_i)

In [ ]:
soma(0.5).hh.gkbar = 0.0
soma(0.5).K_Tst.gK_Tstbar = 0.0
#soma(0.5).Kv4_2_0016.gKv4_2bar = 0
soma(0.5).hh.gnabar = 0.0 # Zero HH Na conductance
fig = plt.figure()
ax1, ax2 = fig.subplots(2, 1)
#for holding_voltage in [-70, -60, -50, -40, -30, -20, -10, 0]:
holding_voltage = 0
for temp in [15, 25, 35]:
    a_t, v, i = run_sim2(0, temp)
    ax1.plot(a_t, i, lw=1, label="%.1f deg C" % temp)
    ax2.plot(a_t, v, lw=1, label="%f mV" % holding_voltage)
    ax1.legend()
    #plt.legend(fontsize=12)
ax1.set_xlabel("t [ms]", size=16)
ax1.set_ylabel("i [nA]", size=16)
ax2.set_xlabel("t [ms]", size=16)
ax2.set_ylabel("v [mV]", size=16)
ax1.axis([195,225,-0.1,0.6])
ax2.axis([195,225, -80, 20])